<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/10_document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文書分類

* IMDBデータセットを使った感情分析
  * 映画レビューを、映画に肯定的か否定的かで、2値に分類する。

* 文書分類の目的
  1. 検証データでできるかぎりチューニングを行い、最後にテストデータでの分類性能を明らかにする。

  2. 肯定的なレビューと否定的なレビューとを分類する際に、どのような単語が特に効いているか、明らかにする。

## 準備

### Hugging Faceの`datasets`ライブラリのインストール
* 今回は、IMDBデータセットを取得するためだけに使う。

In [ ]:
!pip install datasets

### IMDBデータセットの取得

In [ ]:
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/imdb")

In [ ]:
dataset["train"]

In [ ]:
dataset["train"]["text"][0]

In [ ]:
dataset["train"]["label"][:10]

In [ ]:
dataset["test"]

* 後で扱いやすいようにNumPyの配列に変換しておく。

In [ ]:
import numpy as np

train_corpus = np.array(dataset["train"]["text"])
y_train = np.array(dataset["train"]["label"])

test_corpus = np.array(dataset["test"]["text"])
y_test = np.array(dataset["test"]["label"])

### インポート

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
from sklearn.feature_extraction.text import TfidfVectorizer

%config InlineBackend.figure_format = 'retina'

## TF-IDFによるテキストの埋め込み



### TF-IDFとは
* TF-IDFはテキストをベクトル化する古典的な方法。
* ベクトルの次元は語彙数となる。

  * すべての単語について、テキストごとに重みを計算する方法。
  * TFとは、あるテキストのなかにその単語が何回出現するか、その回数。
  * DFとは、単語が出現するテキストの数。IDFはDFの逆数。
  * TF-IDFは、ざっくり言うと、TFとIDFの積。
* 特定のテキストに注目すると・・・
  * そのテキストに出現する回数が多い単語ほど、TF-IDFは大きい。
  * しかし、多数のテキストに出現する単語は、IDFが小さくなるので・・・
  * そのテキストで頻出していても、TF-IDFは小さくなる。
* scikit-learnの`TfidfVectorizer`で簡単に計算できる。

### `TfidfVectorizer`の使い方
* 訓練セットだけでfitすること。検証セットやテストセットはtransformするだけ。

* `stop_words='english'`とは、英語のストップワードは語彙から取り除く、という意味。
  * ストップワードとはthe, a, is, ofなど、ありふれすぎていて、文書分類などのタスクにはあまり効かない単語のこと。こういう単語は、特徴量削減の意味も含めて、しばしばあらかじめ削除しておく。

* `min_df`は、0から1の間の実数で指定すると、その割合より少ない文書にしか出現しない単語を削除する、という意味のパラメータ。
  * 希少な単語を削除するために使う。

* `max_df`は、0から1の間の実数で指定すると、その割合より多い文書に出現する単語を削除する、という意味のパラメータ。
  * ありふれた単語を削除するために使う。

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(dataset["train"]["text"])
print('X_train: 文書数 {}, 語彙数 {}'.format(*X_train.shape))

* 語彙を取得する。


In [ ]:
vocab = np.array(vectorizer.get_feature_names_out())

* 語彙の一部を見てみる（アルファベット順に並んでいるようだ）。



In [ ]:
print(vocab[500:520])

* 最初の文書をTF-IDFでベクトル化したらどうなったかを、見てみる。
  * スパースな表現になっている。



In [ ]:
print(type(X_train))
print(X_train[0])

## ロジスティック回帰のチューニング

### L2正則化
* 交差検証で正則化パラメータをチューニングする。

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')

for C in 10. ** np.arange(-1, 4):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
  scores = []
  for train_index, valid_index in skf.split(train_corpus, y_train):
    X_train = vectorizer.fit_transform(train_corpus[train_index])
    clf = LogisticRegression(C=C, solver='liblinear', random_state=123)
    clf.fit(X_train, y_train[train_index])
    X_valid = vectorizer.transform(train_corpus[valid_index])
    score = clf.score(X_valid, y_train[valid_index])
    print(f"\t{score:.3f}", end=" ")
    scores.append(score)
  print(f"\nmean accuracy: {np.array(scores).mean():.3f} for C={C:.2e}")

### L1正則化
* 交差検証で正則化パラメータをチューニングする。

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')

for C in 10. ** np.arange(-1, 4):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
  scores = []
  for train_index, valid_index in skf.split(train_corpus, y_train):
    X_train = vectorizer.fit_transform(train_corpus[train_index])
    clf = LogisticRegression(penalty='l1', C=C, solver='liblinear', random_state=123)
    clf.fit(X_train, y_train[train_index])
    X_valid = vectorizer.transform(train_corpus[valid_index])
    score = clf.score(X_valid, y_train[valid_index])
    print(f"\t{score:.3f}", end=" ")
    scores.append(score)
  print(f"\nmean accuracy: {np.array(scores).mean():.3f} for C={C:.2e}")

## TF-IDFのチューニング
* `min_df`を変えてみる。
  * `max_df`の設定も変えてよい。

In [ ]:
for min_df in [0.0, 1/5000, 1/2000, 1/1000]:

  vectorizer = TfidfVectorizer(stop_words='english', min_df=min_df)

  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
  scores = []
  for train_index, valid_index in skf.split(train_corpus, y_train):
    X_train = vectorizer.fit_transform(train_corpus[train_index])
    clf = LogisticRegression(C=1.0, solver='liblinear', random_state=123)
    clf.fit(X_train, y_train[train_index])
    X_valid = vectorizer.transform(train_corpus[valid_index])
    score = clf.score(X_valid, y_train[valid_index])
    print(f"\t{score:.3f}", end=" ")
    scores.append(score)
  print(f"\nmean accuracy: {np.array(scores).mean():.3f} for min_df={min_df}")

## テストセットで評価

* LogisticRegression() のカッコ内には、自分で見つけたベストの設定を書き込む。
* 学習は、訓練データ全体（テストデータ以外の全体）を使っている。

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english")
X_train = vectorizer.fit_transform(train_corpus)

clf = LogisticRegression(C=1.0, solver="liblinear", random_state=123)
clf.fit(X_train, y_train)

X_test = vectorizer.transform(test_corpus)
score = clf.score(X_test, y_test)
print(f"test accuracy: {score:.3f}")

In [ ]:
RocCurveDisplay.from_estimator(clf, X_test, y_test, name="logistic regression");

In [ ]:
PrecisionRecallDisplay.from_estimator(clf, X_test, y_test, name="logistic regression");

## SVM

* SVMについては、私からは実行例を示しません。各自、試行錯誤してください。

## ロジスティック回帰による分類に効いている単語を調べる

* 下に示すのは、あくまで一つの方法にすぎないです。他にどんな方法があるか調べて使ってみよう。


### ロジスティック回帰の係数をそのまま可視化する

* 正の係数と負の係数で、それぞれ絶対値が大きいもの30個を可視化する。

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english")
X_train = vectorizer.fit_transform(train_corpus)
vocab = np.array(vectorizer.get_feature_names_out())

clf = LogisticRegression(C=1.0, solver='liblinear', random_state=123)
clf.fit(X_train, y_train)

n_features = 30
positions = np.arange(n_features)[::-1]

* 肯定的なレビューへの分類に効いている単語

In [ ]:
indices = np.argsort(- clf.coef_[0])[:n_features]
widths = clf.coef_[0][indices]
yticks = vocab[indices]

plt.figure(figsize=(12,9))
plt.barh(positions, widths, align='center')
plt.yticks(positions, yticks)
plt.xlabel('coefficients')
plt.title(f'Coefficients of {n_features} Features Using Logistic Regression');

* 否定的なレビューへの分類に効いている単語

In [ ]:
indices = np.argsort(clf.coef_[0])[:n_features]
widths = clf.coef_[0][indices]
yticks = vocab[indices]

plt.figure(figsize=(12,9))
plt.barh(positions, widths, align='center')
plt.yticks(positions, yticks)
plt.xlabel('coefficients')
plt.title(f'Coefficients of {n_features} Features Using Logistic Regression');

## SVMによる分類に効いている単語を調べる

* SVMについては実行例を示しません。上を参考に各自試してみてください。

# 課題
* SVMによる分類（上で空欄にしてある部分）と分類器の解釈を実践してみよう。